<a href="https://colab.research.google.com/github/thisiscam/tuvvg/blob/main/tuvvg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TUVVG: The Ultimate Vase-mode Vase generator

This scripts is a powerful generator for **any** vase-mode vase mesh object.

Simply define your own side surface function and get a 3D printable vase!

In [3]:
!pip install numpy-stl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Core function `make_vase` 

Expand to see the documentation, or skip to next section for sample usage) 

In [5]:
from typing import Callable

import numpy as np
import scipy.spatial
import stl

SurfaceFn = Callable[[np.ndarray, np.ndarray], np.ndarray]


def make_vase(surface_fn: SurfaceFn, npoints: int = 100) -> stl.mesh.Mesh:
    """Make a vase mesh from a function that draws its side surface.
    
    This function takes in a function that draws a side surface of a vase 
    (cylinder-like object) and completes it into a mesh by adding the the top and 
    bottom surface.
    The constructed mesh can be directly used in most slicers that supports 
    "vase mode printing".
    In principle, this function can be used to draw _any_ 3d object that can 
    be printed in vase mode by being smart about the surface function.

    Args:
        surface_fn: A function that takes two arrays of angles and heights and 
            returns an array of radii. 
        npoints: Number of points to use. You can imagine unraveling a cylinder 
            into a rectangular grid. The parameter `npoints` specifies the 
            number of points in each direction of that grid. The default value
            of 100 is usually good for a quick overview of the mesh, but you 
            may want to increase it for a more detailed mesh prior to printing.
    
    Returns:
        A stl.mesh.Mesh object.
    """
    thetas = np.linspace(0, 2 * np.pi, npoints)
    maxh = 1
    hs = np.linspace(0, maxh, npoints)
    points = np.stack(np.meshgrid(thetas, hs), axis=2).reshape(-1, 2)
    tri = scipy.spatial.Delaunay(points)
    radius = surface_fn(points[..., 0], points[..., 1])

    # side surface
    x, y, z = radius * np.cos(points[:, 0]), radius * np.sin(points[:, 0]), points[:, 1]
    xyz = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T
    xyz_grid = xyz.reshape(npoints, npoints, 3)
    # bottom surface
    bottom_loop = xyz_grid[0, :, :]
    bottom_simplices = np.stack([
            bottom_loop, 
            np.roll(bottom_loop, 1, axis=0), 
            np.zeros((npoints, 3))
        ], axis=1)
    # top surface
    top_loop = xyz_grid[-1, :, :]
    top_simplices = np.stack([
            np.roll(top_loop, 1, axis=0), 
            top_loop, 
            np.broadcast_to(np.array([0, 0, maxh]), (npoints, 3))
        ], axis=1)

    # construct mesh
    data = np.zeros(
        len(tri.simplices) + len(bottom_simplices) + len(top_simplices), 
        dtype=stl.mesh.Mesh.dtype)
    data['vectors'][:len(tri.simplices)] = xyz[tri.simplices.flatten()].reshape(-1, 3, 3)
    data['vectors'][len(tri.simplices):len(tri.simplices) + len(bottom_simplices)] = bottom_simplices
    data['vectors'][len(tri.simplices) + len(bottom_simplices):] = top_simplices
    data['vectors'] *= 100 # unit is mm
    return stl.mesh.Mesh(data.copy())

## Example Usages

In [8]:
from google.colab import files

def surface1(theta, h):
    return 0.25 * (
        np.maximum(np.sin(8 * theta + 3 * h), 0.93) + 
        np.sin(7 * theta + 3 - 3 * (h - 0.1) ** 2)  / 10 + 
        (h - 1 * 2 / 3) ** 2 / 5
    )

def surface2(theta, h):
    return 0.25 * (
        np.maximum(np.sin(6 * (theta + h)), 0.95) + 
        np.sin(7 * theta + 3 - 8 * (h - 0.1) ** 2) / 10 -
        np.minimum(np.abs(np.sin(h * 10) / 30), 0.5))

def download_vase(surface_fn, preview: bool = True):
  file_path = "model.stl"
  npoints = 100 if preview else 500
  vase = make_vase(surface_fn=surface1, npoints=npoints)
  vase.save(file_path)
  files.download(file_path)

download_vase(surface1)
# download_vase(surface2)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>